## Dynamic Programming: Text Justification

In [1]:
def pad(line: list, width: int):
    """
    Pads a line of words with spaces so that the returned line (str) has the given width.
    
    pad(['Hello,', 'my', 'name', 'is', 'Jacob'], 30) = 'Hello,   my   name   is  Jacob'
    pad(['Hello,', 'my', 'name', 'is', 'Jacob'], 50) = 'Hello,        my        name        is       Jacob'
    pad([], 20) = '                    '
    """
    if len(line) == 0:
        return ' ' * width
    elif len(line) == 1:
        word = line[0]
        return word + ' ' * (width - len(word))
    text_width = sum([len(word) for word in line])
    space_width = width - text_width
    num_spaces = len(line) - 1
    width_per_space = int(space_width / num_spaces)
    extra_spaces = space_width % num_spaces
    result = ''
    for i in range(len(line) - 1):
        result += line[i]
        result += ' ' * width_per_space
        if i < extra_spaces:
            result += ' '
    result += line[-1]
    return result

In [2]:
def cost(line: list, width: int):
    """
    Provides the cost of a line of text as a function of desired line width.
    
    cost(['Hello,', 'my', 'name', 'is', 'Jacob'], 30) = 7
    cost(['Hello,', 'my', 'name', 'is', 'Jacob'], 50) = 27
    cost([], 50) = 50
    """
    if len(line) == 0:
        return width
    text_width = sum([len(word) for word in line]) + len(line) - 1
    if text_width > width:
        return float('inf')
    else:
        return width - text_width

The general approach is to find the splitting point for each starting index that yields the minimum total of the current line cost and subsequent line costs. As a DP formulation,

$$\textrm{DP}(i)=\min([\textrm{badness}(i,j)+DP(j) \textrm{ for } j\textrm{ in range}(i+1,n+1)]$$

where $n$ is the number of words in the input string. Notice that we need to store the index of the minimum-cost split for each starting index, or else we will be unable to produce the minimum-cost split.

Also notice that this approach is $O(n^2)$, but we can substantially improve the running time by not considering splits that generate lines longer than the allowable width.

In [3]:
def best_split(words: list, width: int, start: int, cost_dict: dict):
    """
    Returns a tuple, (min_cost, min_index), where min_cost is the minimum total cost
    attainable by breaking a new line starting at the supplied start index, prioritizing
    later splits whenever there are multiple possible splits with the same cost.
    
    best_split(['Hello', 'my', 'name', 'is', 'Jacob.', 'How', 'are', 'you?'], 20, 0, {}) = (5,4)
    This means that the best split is at word 4 and has cost 5:
    'Hello my name is'    -> cost = 4 (20 - 16)
    'Jacob. How are you?' -> cost = 1 (20 - 1)
                    -> total cost = 5
    """
    if start >= len(words):
        return (0, None)
    if start in cost_dict.keys():
        return cost_dict[start]
    min_cost = float('inf')
    min_index = None
    for curr_index in range(start+1, len(words) + 2):
        line_cost = cost(words[start:curr_index], width)
        if line_cost == float('inf'): # stop once a split would generate a line longer than width
            break
        current_cost = line_cost + best_split(words, width, curr_index, cost_dict)[0]
        if current_cost <= min_cost:
            min_cost = current_cost
            min_index = curr_index
    cost_dict[start] = (min_cost, min_index)
    return (min_cost, min_index)

In [4]:
def justify(text: str, width: int):
    """
    Justifies a string of text so that columns have equal width and whitespace is minimized.
    The final line is not justified, assuming it is of length less than the given width.
    
    For example, on the input string
      S = 'Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas.
           Mauris rhoncus non dolor eu maximus.'
    justify(S, 40) returns
       'Pellentesque  habitant  morbi  tristique
        senectus  et netus et malesuada fames ac
        turpis egestas. Mauris rhoncus non dolor
        eu maximus.''
    """
    cost_dict = {}
    text_split = text.split(' ')
    _ = best_split(text_split, width, 0, cost_dict) # store the best split indices in cost_dict
    index = 0
    n = len(text_split)
    justified_text = ''
    while index < n:
        next_index = cost_dict[index][1]
        line = text_split[index:next_index]
        if next_index < n:
            justified_text += pad(line, width) + '\n'
        else:
            justified_text += ' '.join(line) # don't spread out the last line
        index = next_index
    return justified_text

We can compare the result from justifying text with DP to that from using a greedy strategy. The difference is especially noticeable with small column widths:

In [24]:
def greedy_justify(text: str, width: int):
    justified_text = ''
    current_line = []
    current_length = 0
    for word in text.split(' '):
        if current_length == 0:
            current_line.append(word)
            current_length += len(word)
        elif current_length + len(word) + 1 < width:
            current_line.append(word)
            current_length += len(word) + 1
        else: # line complete
            justified_text += pad(current_line, width) + '\n'
            current_line = [word]
            current_length = len(word)
    if len(current_line) != 0:
        justified_text += ' '.join(current_line) # don't spread out the last line
    return justified_text

In [26]:
with open('blog_text.txt') as file:
    text = file.read().replace('\n', ' ').replace('  ', ' ')
    DP_justified_text = justify(text, 30)
    print('------DP Justified Text-------', DP_justified_text, sep='\n')
    greedy_justified_text = greedy_justify(text, 30)
    print('\n\n-----Greedy Justified Text----', greedy_justified_text, sep='\n')

------DP Justified Text-------
Certain  concepts  we  covered
were  totally  foreign  to me,
especially those corresponding
to    older    paradigms   and
languages.     Fortran,    for
example,  did  not  originally
support  recursion because all
variables    were    allocated
statically.  For  that matter,
programming  with  no  dynamic
data  (no  heap)  was at times
awkward-and   at  other  times
extremely  elegant, as was the
case  with  a number of Prolog
programs I wrote. (Indeed, the
declarative        programming
paradigm    in   general   was
unfamiliar.)   Never   had  it
occurred   to   me  that  some
programming  languages did not
(and  still  do  not)  support
exception handling. I also did
not    previously   have   any
understanding      of      how
object-oriented programming is
implemented    in    C++,   so
learning  about virtual tables
and  pointers  was  especially
interesting.  In  fact, we had
to   translate  a  class-based
Java program into C-and I will
never  a